In [2]:
# Turing Machine to calculate Fibonacci Sequence
# input = n
# output = F(n)

In [186]:
def parse(defn):
    states = {}
    for s in defn:
        state_name = s.split(":")[0]
        t_string = s.split(":")[1].strip().split(" ")
#         print(t_string)
#         assert (len(t_string) == 3)
        transition = {}
        for t in t_string:
            read = t[0]
            write = t[1]
            direction = t[2]
            next_state = t[3:]
            transition[read] = {"write": write, "direction": direction, "next":next_state}
        states[state_name] = transition
    return states

def run(machine, tape, debug=False):
    state = "START"
    head = 0
    head_char = None
    write = None
    while state != "HALT":
        head_char = tape[head]
        write = machine[state][head_char]["write"]
        tape[head] = write
        direction = machine[state][head_char]["direction"]
        next_state = machine[state][head_char]["next"]
        
        if debug == True:
            print_tape(tape, head)
            print("state ", state, "read", head_char, "write" , write)
            
        if direction == "R":
            head += 1
            if head >= len(tape):
                tape.append("_")
        else:
            head -= 1
            if head < 0:
                tape.insert(0,"_")
                head += 1
                
        state = next_state
        
    if debug == True:
        print_tape(tape, head)
        print("state ", state)
        
    return tape

def print_tape(tape, head):
    for i in range(0,head):
        print(tape[i], end="")
    print("^", end="")
    for i in range(head, len(tape)):
        print(tape[i], end="")
    print()
    

In [181]:
#Format: State number: (input<0,1,_>,output<0,1,_>,head<L,R>,newstate) x 3
#Machine: reads input, writes output to tape, moves in the direction specified
#Go to the end of the input
machine = [
    "START: 00RSTART 11RSTART __Rinit0",
    #write 00_01_ (we are using 00 to represent number 0 and 01 to represent 1 b/c we need to mark chars as read or not)
    "init0: 00Rinit1 10Rinit1 _0Rinit1",
    "init1: 00Rinit2 10Rinit2 _0Rinit2",
    "init2: 0_Rinit3 1_Rinit3 __Rinit3",
    "init3: 00Rinit4 10Rinit4 _0Rinit4",
    "init4: 01Rinit5 11Rinit5 _1Rinit5",
    "init5: 0_Lback0 1_Lback0 __Lback0", #moving L because we wrote a 1 already, so now we want to go back
    # tape now is "n_0*_1", go back to n
    "back0: 00Lback0 11Lback0 __Lback1",
    "back1: 00Lback1 11Lback1 __Lback2",
    "back2: 00Lback2 11Lback2 __Rcheckn0", #moving R because we were at a blank (to the left of n)
    #back at beginning, check to see if n == 0
    "checkn0: 00Rcheckn0 11Rsub0 __Rclean",
    # state 7 means n != 0
    # move to the right of n
    "sub0: 00Rsub0 11Rsub0 __Lsub1",
    # now at least significant bit of n, subtract 1
    "sub1: 01Lsub1 10Lsub2 __RHALT", # should never reach _
    "sub2: 00Lsub2 11Lsub2 __Rfib0",
    # state clean means n == 0  
    "clean: 00LHALT 11LHALT __LHALT",
    # state fib0 means we have to start adding the stuff (we are at leftmost bit of n)
    "fib0: 00Rfib0 11Rfib0 __Rfib1",
    "fib1: 00Rfib1 11Rfib1 __Rfib2", # at leftmost bit of the first number
    "fib2: 00Rfib2 11Rfib2 __Rfib3",
    "fib3: 00Rfib3 11Rfib3 _1Lfib4", # for now just keep on adding 1's to the end of the list (eventually do the sum)
    "fib4: 00Lfib4 11Lfib4 __Lback0", # we need to go back one extra "_" before going to the back subroutine
]

In [182]:
mach = parse(machine)
run(mach, tape = list("1011"), debug=True)

^1011
state  START read 1 write 1
1^011
state  START read 0 write 0
10^11
state  START read 1 write 1
101^1
state  START read 1 write 1
1011^_
state  START read _ write _
1011_^0
state  init0 read _ write 0
1011_0^0
state  init1 read _ write 0
1011_00^_
state  init2 read _ write _
1011_00_^0
state  init3 read _ write 0
1011_00_0^1
state  init4 read _ write 1
1011_00_01^_
state  init5 read _ write _
1011_00_0^1_
state  back0 read 1 write 1
1011_00_^01_
state  back0 read 0 write 0
1011_00^_01_
state  back0 read _ write _
1011_0^0_01_
state  back1 read 0 write 0
1011_^00_01_
state  back1 read 0 write 0
1011^_00_01_
state  back1 read _ write _
101^1_00_01_
state  back2 read 1 write 1
10^11_00_01_
state  back2 read 1 write 1
1^011_00_01_
state  back2 read 0 write 0
^1011_00_01_
state  back2 read 1 write 1
^_1011_00_01_
state  back2 read _ write _
_^1011_00_01_
state  checkn0 read 1 write 1
_1^011_00_01_
state  sub0 read 0 write 0
_10^11_00_01_
state  sub0 read 1 write 1
_101^1_00_01_
state 

state  fib4 read 1 write 1
_0100_00_01^_1111111
state  fib4 read _ write _
_0100_00_0^1_1111111
state  back0 read 1 write 1
_0100_00_^01_1111111
state  back0 read 0 write 0
_0100_00^_01_1111111
state  back0 read _ write _
_0100_0^0_01_1111111
state  back1 read 0 write 0
_0100_^00_01_1111111
state  back1 read 0 write 0
_0100^_00_01_1111111
state  back1 read _ write _
_010^0_00_01_1111111
state  back2 read 0 write 0
_01^00_00_01_1111111
state  back2 read 0 write 0
_0^100_00_01_1111111
state  back2 read 1 write 1
_^0100_00_01_1111111
state  back2 read 0 write 0
^_0100_00_01_1111111
state  back2 read _ write _
_^0100_00_01_1111111
state  checkn0 read 0 write 0
_0^100_00_01_1111111
state  checkn0 read 1 write 1
_01^00_00_01_1111111
state  sub0 read 0 write 0
_010^0_00_01_1111111
state  sub0 read 0 write 0
_0100^_00_01_1111111
state  sub0 read _ write _
_010^1_00_01_1111111
state  sub1 read 0 write 1
_01^11_00_01_1111111
state  sub1 read 0 write 1
_0^011_00_01_1111111
state  sub1 read 1 writ

In [183]:
len("11111111111")

11

In [216]:
add = [
    "START: 01Raddnc1 11Raddnc2 __L0nc",
    "addnc1: 00R0nc 11R1nc",
    "addnc2: 00RSTART 11RSTART",
    
    "addc0: 01Raddc1 11Raddc2 __L1nc",
    "addc1: 00R1nc 11R0c",
    "addc2: 00Raddc0 11Raddc0",
    
    "0nc: 00R0nc 11R0nc __R0nc1",
#     "0nc1: 01R0nc2 11R0nc3 __Lwrite0nc",
    "0nc1: 01R0nc2 11R0nc3 __Rfinish",
    "0nc2: 00Rwrite0nc 11Rwrite1nc",
    "0nc3: 00R0nc1 11R0nc1",
    
    "1nc: 00R1nc 11R1nc __R1nc1",
    "1nc1: 01R1nc2 11R1nc3 __Lwrite1nc",
#     "1nc1: 01R1nc2 11R1nc3 __RHALT",
    "1nc2: 00Rwrite1nc 11Rwrite0c",
    "1nc3: 00R1nc1 11R1nc1",
    
    "0c: 00R0c 11R0c __R0c1",
#     "0c1: 01R0c2 11R0c3 __Lwrite0c",
    "0c1: 01R0c2 11R0c3 __Rfinish",
    "0c2: 00Rwrite0c 11Rwrite1c",
    "0c3: 00R0c1 11R0c1",
    
    "write0nc: 00Rwrite0nc 11Rwrite0nc __Rwrite0nc1",
    "write0nc1: 00Rwrite0nc1 11Rwrite0nc1 _0Rwrite0nc2",
    "write0nc2: 00Rwrite0nc1 11Rwrite0nc1 _0Lgobacknc",
#     "write0nc1: _0Rwrite0nc2",
#     "write0nc2: _0Lgobacknc",
    
    "write1nc: 00Rwrite1nc 11Rwrite1nc __Rwrite1nc1",
    "write1nc1: 00Rwrite1nc1 11Rwrite1nc1 _0Rwrite1nc2",
    "write1nc2: 00Rwrite1nc1 11Rwrite1nc1 _1Lgobacknc",
#     "write1nc1: _0Rwrite1nc2",
#     "write1nc2: _1Lgobacknc",
    
    "write0c: 00Rwrite0c 11Rwrite0c __Rwrite0c1",
    "write0c1: 00Rwrite0c1 11Rwrite0c1 _0Rwrite0c2",
    "write0c2: 00Rwrite0c1 11Rwrite0c1 _0Lgobackc",
#     "write0c1: _0Rwrite0c2",
#     "write0c2: _0Lgobackc",
    
    "write1c: 00Rwrite1c 11Rwrite1c __Rwrite1c1",
    "write1c1: 00Rwrite1c1 11Rwrite1c1 _0Rwrite1c2",
    "write1c2: 00Rwrite1c1 11Rwrite1c1 _1Lgobackc",
#     "write1c1: _0Rwrite0c2",
#     "write1c2: _1Lgobackc",
    
    "gobacknc: 00Lgobacknc 11Lgobacknc __Lgobacknc1",
    "gobacknc1: 00Lgobacknc1 11Lgobacknc1 __Lgobacknc2",
    "gobacknc2: 00Lgobacknc2 11Lgobacknc2 __RSTART",
#     "gobacknc3: 00Lgobacknc3 11Lgobacknc3 __LSTART",
    
    "gobackc: 00Lgobackc 11Lgobackc __Lgobackc1",
    "gobackc1: 00Lgobackc1 11Lgobackc1 __Lgobackc2",
    "gobackc2: 00Lgobackc2 11Lgobackc2 __Raddc0",
#     "gobackc3: 00Lgobackc3 11Lgobackc3 __Laddc0",
    "finish: 00Lfinish 11Lfinish __Lfinish1",
    "finish1: 00Lfinish1 11Lfinish1 __Lfinish2",
    "finish2: 00Lfinish2 11Lfinish2 __RHALT",
]

add_mach = parse(add)

In [217]:
run(add_mach, list("0101_0101"), debug=True)
pass

^1101_0101
state  START read 0 write 1
1^101_0101
state  addnc1 read 1 write 1
11^01_0101
state  1nc read 0 write 0
110^1_0101
state  1nc read 1 write 1
1101^_0101
state  1nc read _ write _
1101_^1101
state  1nc1 read 0 write 1
1101_1^101
state  1nc2 read 1 write 1
1101_11^01
state  write0c read 0 write 0
1101_110^1
state  write0c read 1 write 1
1101_1101^_
state  write0c read _ write _
1101_1101_^0
state  write0c1 read _ write 0
1101_1101_0^0
state  write0c2 read _ write 0
1101_1101_^00
state  gobackc read 0 write 0
1101_1101^_00
state  gobackc read _ write _
1101_110^1_00
state  gobackc1 read 1 write 1
1101_11^01_00
state  gobackc1 read 0 write 0
1101_1^101_00
state  gobackc1 read 1 write 1
1101_^1101_00
state  gobackc1 read 1 write 1
1101^_1101_00
state  gobackc1 read _ write _
110^1_1101_00
state  gobackc2 read 1 write 1
11^01_1101_00
state  gobackc2 read 0 write 0
1^101_1101_00
state  gobackc2 read 1 write 1
^1101_1101_00
state  gobackc2 read 1 write 1
^_1101_1101_00
state  goback

In [236]:
def test_add(i, j):
    t = ("0".join(bin((max(i,j)))[2:]) + "0_" + "0".join(bin(min(i,j))[2:]) + "0")[::-1]
    out = run(parse(add), list(t), debug=False)
    num = int(("".join(out)).split("_")[-1][1::2][::-1], 2)
    print("Good" if i + j == num else "BADBADBAD", i, j, num)
    if i+j != num:
        print(t)
        print("".join(out))
        return False
    return True


from random import randint

for i in range(1024):
    if test_add(randint(0,1<<79), randint(0,1<<86)) == False:
        break

Good 562148763386139673938559 22838401535816871540969083 23400550299203011214907642
Good 248490481276329645922097 16263749954463593409095831 16512240435739923055017928
Good 130605139394414459374680 27294913194953421649874136 27425518334347836109248816
Good 26499386076928244976138 68639400398089065831138239 68665899784165994076114377
Good 12657037560786834539293 4069712522460611732239762 4082369560021398566779055
Good 116234785561938578235150 43763006776662144477176749 43879241562224083055411899
Good 224721113656120570610179 56175876810067310034471802 56400597923723430605081981
Good 183196463808163349704193 45184642865636999326522424 45367839329445162676226617
Good 327436763225758797455695 73145782120531676703484784 73473218883757435500940479
Good 81800281752040515469689 62437937231113953480456837 62519737512865993995926526
Good 235428641212154132997161 30242898717700954302973962 30478327358913108435971123
Good 576139847092437859398116 6064539742653355067525345 6640679589745792926923461

Good 579486393516854380291823 61719916683555191689830716 62299403077072046070122539
Good 456419395480719612007148 46735395792754628536243116 47191815188235348148250264
Good 264294562064669771217401 69902921935255577816776099 70167216497320247587993500
Good 336007301707438876380601 52919597740203289995165114 53255605041910728871545715
Good 270564273066998117360157 59909556540064608205793053 60180120813131606323153210
Good 540434049179637773944801 66470798137455573714550330 67011232186635211488495131
Good 224742371788403473710314 8659342592323216799575538 8884084964111620273285852
Good 530820725832103409037109 57980623087036009902156288 58511443812868113311193397
Good 203436373281815016163003 2311827851789397054352558 2515264225071212070515561
Good 401435576766522147919009 36175693400774270472139554 36577128977540792620058563
Good 31929823468654518303069 76498950482663904412873288 76530880306132558931176357
Good 233314717286203100720140 98427172356708347202421 331741889642911447922561
Go

Good 370663388653265106378407 69388523208972619455246207 69759186597625884561624614
Good 371084591687139699311756 34005737827636293281178682 34376822419323432980490438
Good 389090955061549136710857 51634099802283368339392558 52023190757344917476103415
Good 116585752351485495537041 69569146142527814970718838 69685731894879300466255879
Good 573903814285727770166140 66577739108522750670946559 67151642922808478441112699
Good 227664893039498201122270 52425015275895330400798155 52652680168934828601920425
Good 273301261150509862815277 71545243681058189600811490 71818544942208699463626767
Good 177357435871839851010247 31114972215174918206307473 31292329651046758057317720
Good 200292898314734702179558 56836032516772761955599147 57036325415087496657778705
Good 203893351253456549658639 51364510383542554689289524 51568403734796011238948163
Good 46181133081421478133081 3049013092431986529893700 3095194225513408008026781
Good 474141023908045079224616 14365888215722063040899160 1484002923963010812012

Good 526728571045133157418754 61827231701450579043170698 62353960272495712200589452
Good 21048070488388362685138 74763145954102924545729065 74784194024591312908414203
Good 138031808378963869332416 76334223471532593210576681 76472255279911557079909097
Good 97783407444254379464607 43555012214704475956424221 43652795622148730335888828
Good 10253415446082779860492 5060234983768344766746590 5070488399214427546607082
Good 407252133296338096063161 73299011701837179571712492 73706263835133517667775653
Good 187698266554228192001311 31377900704145922076706773 31565598970700150268708084
Good 142825765773862374252677 11979362989314844630431326 12122188755088707004684003
Good 152163110762338589627783 65704947638074184763575002 65857110748836523353202785
Good 103530384801137155968510 44440772158349386573037771 44544302543150523729006281
Good 497780606107635107078886 12094888470089843992672740 12592669076197479099751626
Good 412574961605220012389236 64453658012053589053078439 648662329736588090654676

Good 534471864380262761698386 66141669195536503551120418 66676141059916766312818804
Good 484612611754128792410775 74692434673506101681827361 75177047285260230474238136
Good 352239994787774314027915 22400032004964968795757263 22752271999752743109785178
Good 222052029490004751380663 37221025732614724346645437 37443077762104729098026100
Good 586488972966953588578904 52059809246493893561933667 52646298219460847150512571
Good 134081976367769281545811 14005365278229228242124885 14139447254596997523670696
Good 513698290678789023919593 19401202907546671035776041 19914901198225460059695634
Good 507340585697100602869197 1542977247883918901716112 2050317833581019504585309
Good 519510949431054467282829 36472418878860799136574842 36991929828291853603857671
Good 243633585287280721037778 69181590093800230615917444 69425223679087511336955222
Good 10329938525340708960580 76364092945138252786025429 76374422883663593494986009
Good 411795041223688218159520 53835549271893124472104750 5424734431311681269026

Good 592384218108240593487825 27674473288491936722103820 28266857506600177315591645
Good 426258577524637225909508 52826650255222214819288933 53252908832746852045198441
Good 245256466371607154721872 1009047366964974055064080 1254303833336581209785952
Good 249020759366473993435509 13734810175871832590102495 13983830935238306583538004
Good 418563312880497673403599 43402403701072289139793891 43820967013952786813197490
Good 560410721949359864309 3250546825161155146899740 3251107235883104506764049
Good 598951715250545004526489 12928516006590565736796945 13527467721841110741323434
Good 329124057833952861527113 40471483956305131150591445 40800608014139084012118558
Good 140003698131124892385595 24758344886478932277295913 24898348584610057169681508
Good 88494003055878888989462 62059570296679265353423069 62148064299735144242412531
Good 532204163628024869887420 8288475988998581761014970 8820680152626606630902390
Good 168435102773744646014297 23910491150554911869588482 24078926253328656515602779
Go

Good 501528991226054669592198 9936156742096874664429258 10437685733322929334021456
Good 323773861931421920397269 29674892157720488460786923 29998666019651910381184192
Good 382277906653988472909114 42070447526968598034889348 42452725433622586507798462
Good 384803618326695168380016 56638101522500367662671028 57022905140827062831051044
Good 550241524696124077640185 56408468945787368718369740 56958710470483492796009925
Good 502911993851504239964347 32958582432673037527603282 33461494426524541767567629
Good 489434869117197120844274 1263395164432081816650193 1752830033549278937494467
Good 418457053215024092390307 70150351876920737322833160 70568808930135761415223467
Good 30187044454972912008241 74570085764644621711481966 74600272809099594623490207
Good 499852633324740151070987 74377697001483044416635068 74877549634807784567706055
Good 260004859023967534008500 12246913845401250999701046 12506918704425218533709546
Good 156211238858201470058936 73485537698342739615311440 73641748937200941085370

Good 152654918883057434216830 4138465012414773543033875 4291119931297830977250705
Good 523299054816583220634051 6104072312060077232828639 6627371366876660453462690
Good 35353366147942143250976 11181018684070110364216839 11216372050218052507467815
Good 84044723125866525340474 73154555832544402329502150 73238600555670268854842624
Good 406408874519593191532777 10204679055156096240967068 10611087929675689432499845
Good 342853176427560112219131 17716268215038272127653430 18059121391465832239872561
Good 344283233082154856591198 59031027998968357595659068 59375311232050512452250266
Good 240735217435335554796096 57272680472493511299573603 57513415689928846854369699
Good 89625471303798551787833 52457210682288108272301366 52546836153591906824089199
Good 206669353622439571112030 71583770505127881145888923 71790439858750320717000953
Good 290243864398432428759160 68172626914904614286876673 68462870779303046715635833
Good 498009261236305232178869 19094927821228567422217907 19592937082464872654396776

Good 172045090956979116383403 12172448558003293030299531 12344493648960272146682934
Good 559280819816686887442794 50639927389524135447685833 51199208209340822335128627
Good 165596513837487153249996 17466184640468857345374822 17631781154306344498624818
Good 492218261022994611357735 39272417152454513873601797 39764635413477508484959532
Good 437384998192635616860725 66660229776103555138133463 67097614774296190754994188
Good 548343786077134679458747 63001403053296910922682171 63549746839374045602140918
Good 195347085973481708915603 67535736468313817152762497 67731083554287298861678100
Good 112524723442413197373215 21129209275289035423404995 21241733998731448620778210
Good 586320547503800555548340 30443676566669740585966544 31029997114173541141514884
Good 588664177759281227037719 72300638551117405799037000 72889302728876687026074719
Good 446770355226111663869306 50419089667383351277247660 50865860022609462941116966
Good 507280897801077540882552 51444879841248914522560775 5195216073904999206

Good 165158035949522154597642 28907273016847762904071244 29072431052797285058668886
Good 312172295362300768508825 64020112798270344588361838 64332285093632645356870663
Good 324001324542496009425469 73418274441186237532106296 73742275765728733541531765
Good 151861050008697743997209 73978367247369161851832149 74130228297377859595829358
Good 513951347430921285010646 29060604009853731840510798 29574555357284653125521444
Good 459551662877249082379690 56263091762361421653008753 56722643425238670735388443
Good 123582210431965675025256 12233234264563957999298147 12356816474995923674323403
Good 280453283591369129156988 43238427913603721397963915 43518881197195090527120903
Good 415592946447229672292457 47759445802449199084841491 48175038748896428757133948
Good 101443852899859254950203 46319592200748988453984657 46421036053648847708934860
Good 158413255923134656246669 18457496313770229654118557 18615909569693364310365226
Good 7102118788291300356965 57739800635258223483003739 577469027540465147833